<a href="https://colab.research.google.com/github/deepu-san/Projects/blob/main/Projects/Amaon%20Product%20Review/Amazon_Prod_review_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from scipy import stats

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [2]:
path="https://github.com/deepu-san/Projects/blob/main/Amaon%20Product%20Review/Flipkart_Amazon%20Mobile%20Reviews.csv?raw=true"
df = pd.read_csv(path)
df.head(3)
#df.shape

,Unnamed: 0,Review-Title,rating,Review-Body,Product Name
0,0,Worst phone ever,1.0 out of 5 stars,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."
1,1,Ok !!! Not up to the mark,2.0 out of 5 stars,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."
2,2,Awesome look,5.0 out of 5 stars,Camera is so good n very fast phone back look ...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."


In [3]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df.dtypes

Review-Title    object
rating          object
Review-Body     object
Product Name    object
dtype: object

In [4]:
df.isna().sum()

Review-Title      2
rating            0
Review-Body     617
Product Name      0
dtype: int64

In [5]:
df=df.dropna()
df= df.reset_index(drop=True)


In [6]:
df['rating'].value_counts()

5.0 out of 5 stars    9069
1.0 out of 5 stars    5161
4.0 out of 5 stars    4749
3.0 out of 5 stars    2650
2.0 out of 5 stars    1530
Name: rating, dtype: int64

In [7]:
df['Product Name'].value_counts()

Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)           4450
OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)              4273
Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM                 3597
OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)    3396
Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)            2642
realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)      2100
vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)      1365
Samsung Galaxy M32                                            1336
Name: Product Name, dtype: int64

In [8]:
df['rating_processed'] = (df['rating'].str[:3]).astype('float')
df.head(2)

,Review-Title,rating,Review-Body,Product Name,rating_processed
0,Worst phone ever,1.0 out of 5 stars,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",1.0
1,Ok !!! Not up to the mark,2.0 out of 5 stars,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",2.0


In [9]:
df['rating_nature'] =df['rating_processed'].apply(lambda x: 'Pos' if x>2.5 else 'Neg')
df.head(2)

,Review-Title,rating,Review-Body,Product Name,rating_processed,rating_nature
0,Worst phone ever,1.0 out of 5 stars,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",1.0,Neg
1,Ok !!! Not up to the mark,2.0 out of 5 stars,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",2.0,Neg


In [ ]:
df['review_processed'] = df['Review-Body'].str.replace("[^a-zA-Z0-9]", " ")
df['review_processed'] = df['review_processed'].apply(lambda row: ' '.join([word for word in row.split() if len(word)>2]))
df['review_processed'] = [review.lower() for review in df['review_processed']]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english')


def remove_stopwords(rev):
    # iNPUT : IT WILL TAKE ROW/REVIEW AS AN INPUT
    # take the paragraph, break into words, check if the word is a stop word, remove if stop word, combine the words into a para again
    review_tokenized = word_tokenize(rev)
    rev_new = " ".join([i for i in review_tokenized  if i not in stop_words])
    return rev_new

# Removing stopwords
df['review_processed'] = [remove_stopwords(r) for r in df['review_processed']]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') # Run atleast once 

lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')

# Finds the part of speech tag
# Convert the detailed POS tag into a shallow information
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

# lemmatize sentence using pos tag
def lemmatize_sentence(sentence):
  # word tokenize -> pos tag (detailed) -> wordnet tag (shallow pos) -> lemmatizer -> root word
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)


df['review_processed'] = df['review_processed'].apply(lambda x: lemmatize_sentence(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
pd.set_option("display.max_colwidth", -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
nltk.download('vader_lexicon')

def sentiment_analyse(sentiment_text):
    score = SentimentIntensityAnalyzer().polarity_scores(sentiment_text)
    if score['neg'] > score['pos']:
        return ("Negative")
    elif score['neg'] < score['pos']:
        return ("Positive")
    else:
        return ("Neutral")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
sid = SentimentIntensityAnalyzer()
def comp_score(txt):
    score=sid.polarity_scores(txt)
    if score['compound']>0 : return "Pos"
    else : return "Neg"

In [ ]:
df['review_score']=[sentiment_analyse(r) for r in df['Review-Body']]
#df['review_comp']=[comp_score(r) for r in df['Review-Body']]

In [ ]:
df['review_comp']=[comp_score(r) for r in df['Review-Body']]

In [ ]:
df[['Review-Body','review_score','review_comp']]

,Review-Body,review_score,review_comp
0,Hang problem,Negative,Neg
1,I'm writing this review after using 3days !!!Be to honest this is normal Android phoneIt's not like OnePlusCamera quality very low it says 64mp but notSound also lowBattery 🔋 backup ok !!! For one day normal use itsIf ur Gamer don't go for itOverall price it High for this handsetIf u really wanna buy OnePlus ho for higher end modelI'm first time buying OnePlusLittle disappointment,Negative,Neg
2,Camera is so good n very fast phone back look is awesomeWith dis price segment it's worth it n d most important thing is the phone has dedicated memory card slot I think dis is the first one plus phone which has d dedicated slot for memory cardBattery backup is also good n wit the help of 65w charger u can charge ur phone frm 50-100% in just 15 minsIt has the super vooc charger frm Dat u can charge ur phone very fastVery smooth touch n very fast phone u can play games with no lagInternet connectivity is also fast n lightning wit my Jio sim as u can c I hav pasted one photo showing d downloading speed of 11mbps wit my cellular networkI think everyone who wants to buy dis phone just go for itI think oneplus launched dis phone with lots of good features n d most important thing is price is very lessI recommendJust go for itThank u oneplus n Amazon for dis wonderful product,Positive,Pos
3,The media could not be loaded.\n \n\n\n\n It’s A average product decent for day to day activity.. one the thing I would like to mention that exchange was Far Far alot Good On amazon it was like 2 to 3 minutes and delivery agent was very Good with me although I had 2 scratches on it but he neglected it as it was minor scratches and I exchanged poco m2 pro 6 64gb variant which was exchanged for rupees 10100. I bought it at 12499 1 and Half year ago and sold it at at very good price!! So plus point To amazonComing Back to device I would say it’s a ok device camera is average performance is Fine one plus is losing it’s originality oxygen os to color or..WARP CHARGER TO VOOC CHARGER!! yes it has been taken by oppo!! I wish the way we believed in one plus doesn’t change with it as A one plus Fan We don’t want To merge with Other brands please keep your Originality Maintained!! thanx amazon for such a hasselfree exchange alot better than FLIPKART,Positive,Pos
4,I got my delivery on 23 feb when I unboxed the phone the display was damaged. I requested for replacement of my new phone so technician was suppose to come for inspection but instead of a visit he just talked on call and asked me to install an application. He did not even saw the damage of my display screen.Lets hope amazon does it right replace my phone. Will update the rest later.But amazon pls consider my request and replace the item as soon as possibleEdit- The phone is perfect... They replaced it and now everything is perfect...Camera quality is as expected very perfectOverall performance bestYou can go for it,Positive,Pos
...,...,...,...
23154,Must he Bhai iss range me bas cemra thoda sa shi nhi he,Neutral,Neg
23155,Good product,Positive,Pos
23156,Good,Positive,Pos
23157,"Don't buy this mobile phone money weast screen tach problem,no camera quality, no battery 🔋 life, Phone very slow, please don't buy this mobile phone please 🙏🙏🙏🙏🙏",Positive,Pos


In [ ]:
sentiment_analyse("Don't buy this mobile phone money weast screen tach problem,no camera quality, no battery life,Phone very slow,")# please don't buy this mobile phone please ")

'Negative'

In [ ]:
a = 'This was a bad movie.'
sid.polarity_scores(a)

{'compound': -0.5423, 'neg': 0.538, 'neu': 0.462, 'pos': 0.0}

In [ ]:
sid.polarity_scores('The shoes I brought were amazing.')


{'compound': 0.5859, 'neg': 0.0, 'neu': 0.513, 'pos': 0.487}

In [ ]:
sentiment_analyse("please don't buy this mobile phone please")

'Positive'

In [ ]:
sentiment_analyse("The shoes I brought were amazing.")

'Positive'

In [ ]:
comp_score('The shoes I brought were amazing.')

'Pos'

In [ ]:
comp_score("please don't buy this mobile phone please")

'Pos'

In [ ]:
txt1='Must he Bhai iss range me bas cemra thoda sa shi nhi he'

[]